# Importo librerie e pacchetti, carico i dataset

In [ ]:
import pymongo
import pandas as pd
from datetime import datetime

In [ ]:
# Connessione a MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
# Stampa a video della lista dei database

In [ ]:
import os
os.chdir('C:/Users/Laura/Google Drive (l.cavenati1@campus.unimib.it)/MAGISTRALE/2 anno/1 semestre/Data management/progetto data man/Progetto/vario Laura/dataset/')

In [ ]:
calendario = pd.read_csv("CalendarioAzzurriDef.csv", sep=';')
listaazzurri = pd.read_csv("ListaAzzurri.csv")

In [ ]:
import json
with open("parole_sport.json") as f:
    parole_sport = json.loads(f.read())

# Creazione dizionari discipline e atleti
Abbiamo raggruppato gli sport in 33 gruppi come descritto qui https://it.wikipedia.org/wiki/Sport_olimpici. Ogni sport-gruppo (es sport acquatici) è suddivisibile in diverse discipline.
```
len(parole_sport.keys())
```

Siccome nomi diversi nei vari dataset, rinominiamo secondo il nostro standard

In [ ]:
calendario['sport']=calendario['sport'].replace('Arrampicata Sportiva', 'Arrampicata sportiva')
calendario['sport']=calendario['sport'].replace('Atletica', 'Atletica leggera')
calendario['sport']=calendario['sport'].replace('Basket 3x3', 'Pallacanestro')
calendario['sport']=calendario['sport'].replace('Beach Volley', 'Pallavolo')
calendario['sport']=calendario['sport'].replace('Canoa', 'Canoa/kayak')
calendario['sport']=calendario['sport'].replace(['Nuoto', 'Nuoto Sincronizzato', 'Nuoto di Fondo', 'Tuffi', 'Pallanuoto'], 'Sport acquatici')
calendario['sport']=calendario['sport'].replace('Pentathlon Moderno', 'Pentathlon moderno')
calendario['sport']=calendario['sport'].replace('Pesi', 'Sollevamento pesi')
calendario['sport']=calendario['sport'].replace('Skateboarding', 'Skateboard')
calendario['sport']=calendario['sport'].replace('Softball', 'Baseball/softball')
calendario['sport']=calendario['sport'].replace('Sport Equestri', 'Equitazione')
calendario['sport']=calendario['sport'].replace('Tennistavolo', 'Tennis tavolo')
calendario['sport']=calendario['sport'].replace(['Tiro a Segno','Tiro a Volo'], 'Tiro a segno/volo')
calendario['sport']=calendario['sport'].replace('Beach Volley', 'Pallavolo')

In [ ]:
listaazzurri['disciplina']=listaazzurri['disciplina'].replace('Arrampicata Sportiva', 'Arrampicata sportiva')
listaazzurri['disciplina']=listaazzurri['disciplina'].replace('Atletica', 'Atletica leggera')
listaazzurri['disciplina']=listaazzurri['disciplina'].replace('Basket 3x3', 'Pallacanestro')
listaazzurri['disciplina']=listaazzurri['disciplina'].replace('Canoa', 'Canoa/kayak')
listaazzurri['disciplina']=listaazzurri['disciplina'].replace(['Nuoto', 'Nuoto Sincronizzato', 'Nuoto di Fondo', 'Tuffi', 'Pallanuoto'], 'Sport acquatici')
listaazzurri['disciplina']=listaazzurri['disciplina'].replace('Pentathlon Moderno', 'Pentathlon moderno')
listaazzurri['disciplina']=listaazzurri['disciplina'].replace('Pesi', 'Sollevamento pesi')
listaazzurri['disciplina']=listaazzurri['disciplina'].replace('Skateboarding', 'Skateboard')
listaazzurri['disciplina']=listaazzurri['disciplina'].replace('Softball', 'Baseball/softball')
listaazzurri['disciplina']=listaazzurri['disciplina'].replace('Sport Equestri', 'Equitazione')
listaazzurri['disciplina']=listaazzurri['disciplina'].replace('Tennistavolo', 'Tennis tavolo')
listaazzurri['disciplina']=listaazzurri['disciplina'].replace(['Tiro a Segno','Tiro a Volo'], 'Tiro a segno/volo')
listaazzurri['disciplina']=listaazzurri['disciplina'].replace('Beach Volley', 'Pallavolo')

In [ ]:
#creazione del dizionario con i possibili nomi degli atleti
from collections import defaultdict
diz_atleti = defaultdict(dict)

for i in range(len(listaazzurri)):
    diz_atleti[listaazzurri["nome intero"][i]]["nome"] = []
    diz_atleti[listaazzurri["nome intero"][i]]["cognome"] = []
    diz_atleti[listaazzurri["nome intero"][i]]["disciplina"] = []
    for j in range(len(listaazzurri["nome intero"][i].split())):
        if ((len(listaazzurri["nome intero"][i].split()[j])>2) | (listaazzurri["nome intero"][i]=='DAVIDE RE')):
            diz_atleti[listaazzurri["nome intero"][i]]['nome'].append(listaazzurri["nome intero"][i].split()[j])
    diz_atleti[listaazzurri["nome intero"][i]]['disciplina']=listaazzurri["disciplina"][i]
    diz_atleti[listaazzurri["nome intero"][i]]['cognome']=listaazzurri["cognome"][i]
#Davide Re è ambiguo, potrei eliminarlo e bona!!!!!!!!

In [ ]:
#creazione del dizionario con discipline e i nomi dei rispettivi atleti
diz_discipline = {}
for i in range(len(listaazzurri)):
    diz_discipline[listaazzurri["disciplina"][i]] = []
for i in range(len(listaazzurri)):
    for j in range(len(listaazzurri["nome"][i].split())):
        diz_discipline[listaazzurri["disciplina"][i]].append((listaazzurri["nome"][i].split()[j] + " " + listaazzurri["cognome"][i]))

**Mancano gli atleti che praticano: badminton, baseball, calcio, Hockey su prato, Pallamano, rugby perchè l'Italia non ha gareggiato in questi sport.**   
Creo  le chiavi con liste vuote.  

In [ ]:
diz_discipline['Badminton']=[]
diz_discipline['Calcio']=[]
diz_discipline['Hockey su prato']=[]
diz_discipline['Pallamano']=[]
diz_discipline['Rugby']=[]

In [ ]:
for key in parole_sport:
    parole_sport[key]=parole_sport[key]+diz_discipline[key]

In [ ]:
#standardizzazione del formato dell'ora
calendario['inizio']=calendario['inizio']+':00'
calendario['fine']=calendario['fine']+':00'

In [ ]:
#creo dizionario eventi. forse c'è metodo migliore ma già sto impazzendo per fare questo!!!
diz_eventi = {}
for q in range(len(calendario)):
  diz_eventi[calendario['Azzurri in gara'][q]]={}
for q in range(len(calendario)):
  diz_eventi[calendario['Azzurri in gara'][q]][calendario['evento'][q]]={'nome':calendario['evento'][q], 'start':calendario['inizio'][q],  'end':calendario['fine'][q]}

In [ ]:
#questi atleti sono solo in listaazzurri
diz_eventi['MATTEO GALVAN']=[]
diz_eventi['CLARA GUERRA']=[]
diz_eventi['GIORGIO AVOLA']=[]
diz_eventi['ZAYNAB DOSSO']=[]
diz_eventi['MARTINA FIDANZA']=[]
diz_eventi['MARTINA ALZINI']=[]
diz_eventi['STEFANO BRECCIAROLI']=[]
diz_eventi['ALBERTA SANTUCCIO']=[]
diz_eventi['LORENZO BENATI']=[]
diz_eventi['DAVIDE MANENTI']=[]
diz_eventi['RAPHAELA BOAHENG LUKUDO']=[]
diz_eventi['LIAM BERTAZZO']=[]
diz_eventi['LUCA CHIUMENTO']=[]
diz_eventi['AYOMIDE FOLORUNSO']=[]
diz_eventi['JOHANELIS HERRERA ABREU']=[]
diz_eventi['GIUSEPPE LEONARDI']=[]
diz_eventi['HILLARY WANDERSON POLANCO RIJO']=[]
diz_eventi['ANNA POLINARI']=[]

# Funzioni utili

In [ ]:
#funzione che cerca le discipline all'interno dei tweet
def search_discipline(tweet):
    mongo_id=tweet['_id']
    sport_trovati = {sport for sport in parole_sport for parole_da_cercare in parole_sport[sport] if parole_da_cercare.lower() in tweet['text'].lower()}
    discipline = list(sport_trovati)
    tweet['discipline'] = discipline
    if len(tweet['discipline'])>0:
        collection.update_one({'_id': mongo_id},{"$set": {"discipline": tweet['discipline']}})

In [ ]:
cognomi_duplicati=listaazzurri.groupby(['cognome']).count().reset_index()
cognomi_duplicati=cognomi_duplicati[cognomi_duplicati['nome']>1]

In [ ]:
lista_cognomi_duplicati=[cognome for cognome in cognomi_duplicati['cognome']]
lista_cognomi_duplicati.append('RE')

In [ ]:
#funzione che cerca gli atleti all'interno dei tweet
def search_atleti(tweet):
    mongo_id=tweet['_id']
    tweet['atleti']=[]
    try:
        tweet['discipline']
    except:
        tweet['discipline']=[]
    for nomi in diz_atleti:
        count=0
        for nome in diz_atleti[nomi]['nome']:
            if nome.lower() in tweet['text'].lower():
                count+=1
        if (count>=2) | ((diz_atleti[nomi]['cognome'].lower() in tweet['text'].lower()) & (diz_atleti[nomi]['cognome'] not in lista_cognomi_duplicati)):
            tweet['atleti'].append(nomi)
            if diz_atleti[nomi]['disciplina'] not in tweet['discipline']:
                tweet['discipline'].append(diz_atleti[nomi]['disciplina'])
    if len(tweet['atleti'])>0:
        collection.update_one({'_id': mongo_id},{"$set": {"atleti": tweet['atleti']}})
        collection.update_one({'_id': mongo_id},{"$set": {"discipline": tweet['discipline']}})

cosa faccio qua sopra: invece che cercare nome e cognome (o come fa matteo primo/secondo nome + cognome) cerco o solo il cognome (se non ci sono più atleti con lo stesso cognome) o 2 valori tra nomi e cognomi. forse un po' complicato a livello di tempo di esecuzione!!!

In [ ]:
#funzione che cerca gli eventi associati ai tweet
def add_evento(tweet):
    mongo_id=tweet['_id']
    mongo_time=tweet['timestamp']
    tweet['evento']=[]
    try:
        for atleta in tweet['atleti']:
            for evento in diz_eventi[atleta]:
                if (mongo_time>= pd.to_datetime(diz_eventi[atleta][evento]['start'], format='%d/%m/%Y %H:%M:%S')) & (mongo_time<= pd.to_datetime(diz_eventi[atleta][evento]['end'], format='%d/%m/%Y %H:%M:%S')):
                    if diz_eventi[atleta][evento]['nome'] not in tweet['evento']:
                        tweet['evento'].append(diz_eventi[atleta][evento]['nome'])
        collection.update_one({'_id': mongo_id},{"$set": {"evento": tweet['evento']}})
    except:
        pass

# preparazione dataset

In [ ]:
# Get the "Tokyo2020" database
db = client.Tokyo2020
# Get the "totalcollection" collection from the "Tokyo2020" database
collection = db.totalcollection_giusta

Tutti i tweet sono sfalsati(indietro) di due ore. Li traformiamo in oggetti di tipo data e aggiungo 2 ore. Appchiamo le funzioni precedentemente costruite

In [ ]:
%%time
cursore=collection.find({})
for document in cursore:
    mongo_id=document['_id']
    collection.update_one({'_id': mongo_id},{"$set": {"timestamp": pd.to_datetime(document['timestamp'], format='%Y-%m-%d %H:%M:%S')+ pd.Timedelta(hours=2)}})
    collection.update_one({'_id': mongo_id},{"$set": {"timestamp": document['timestamp'].strftime("%Y-%m-%d %H:%M:%S")}})

Wall time: 7min 15s


In [ ]:
%%time
cursore=collection.find({})
for document in cursore:
    search_discipline(document)
#Wall time: 15 min 11s

Wall time: 11min 32s


In [ ]:
%%time
cursore=collection.find({})
for document in cursore:
    search_atleti(document)

Wall time: 24min 43s


In [ ]:
%%time 
cursore=collection.find({})

for document in cursore:
    add_evento(document)

Wall time: 9min
